In [1]:
from xgboost import XGBClassifier, XGBRegressor
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
test_df.head()

,text
0,All the ghetto girls with hot cheeto bags
1,Well we all know which people joins isis
2,White Girl Wednesday 😍 URL
3,"Same with the evil j€ws, they can go back to I..."
4,Why have all channels of communications in Kas...


In [3]:
X = train_df.iloc[:, 0]
y = train_df.iloc[:, 1:]
sentences = X.to_list()
sentences

['This is amazing! Kudos for that women for being so strong',
 '@IlhanMN Omar-you are a racist! Resign! And be deported!',
 "I feel the same. El Paso made me a very open understanding person. I won't judge a person based on their skin color. I will judge someone based on their ignorance and prejudice. None the less I still always try to show everyone love and respect where ever I go.",
 'Stop putting that ugly white person on my tl',
 'Gautam Gambhir of all !!! Lol. Come on !!! Elections in India are conducted and controlled by US and UK. Both INC and BJP are their puppets. Get ready to pay bribes as usual for them to take turns every 10 years to fool and loot you. Ah well lol Sunny Deol too this time, Sunny Leone next.',
 '@ACMichael1 @AmitShah @PMOIndia @NHRCOFINDIA @TelanganaCMO @TelanganaDGP How is that provoking? He is just expressing his concerns over religion conversions. And, FYI this is a very old video taken prior to he becoming an MP.',
 "@realDonaldTrump They won't fix the 

In [4]:
vocab_size = 10000
embedding_dim = 64
max_length = 100
trunc_type = 'post'
oov_token = '<OOV>'
tokenizer = keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
test_sequences = tokenizer.texts_to_sequences(test_df['text'].to_list())
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length)
padded = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length)
test_padded

array([[   0,    0,    0, ...,  765,    1, 4927],
       [   0,    0,    0, ...,   18, 5182,  330],
       [   0,    0,    0, ..., 8254, 1991,   19],
       ...,
       [   0,    0,    0, ...,   24,  858, 4037],
       [   0,    0,    0, ...,   53,   62,    1],
       [   0,    0,    0, ...,    2, 1293, 7359]], dtype=int32)

In [7]:
for column in y.columns:
  print(f'training on {column}')
  model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
  ])
  target = y[column] / 4
  model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.01))
  model.fit(padded, target, epochs=20)
  test_df[column] = model.predict(test_padded)

training on sense
Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


494/494 [==============================] - 9s 12ms/step - loss: 0.6041
Epoch 2/20
494/494 [==============================] - 6s 11ms/step - loss: 0.5542
Epoch 3/20
494/494 [==============================] - 6s 11ms/step - loss: 0.5297
Epoch 4/20
494/494 [==============================] - 6s 11ms/step - loss: 0.5155
Epoch 5/20
494/494 [==============================] - 6s 11ms/step - loss: 0.5033
Epoch 6/20
494/494 [==============================] - 6s 11ms/step - loss: 0.4951
Epoch 7/20
494/494 [==============================] - 6s 11ms/step - loss: 0.4911
Epoch 8/20
494/494 [==============================] - 6s 12ms/step - loss: 0.4857
Epoch 9/20
494/494 [==============================] - 6s 11ms/step - loss: 0.4815
Epoch 10/20
494/494 [==============================] - 6s 11ms/step - loss: 0.4791
Epoch 11/20
494/494 [==============================] - 6s 12ms/step - loss: 0.4775
Epoch 12/20
494/494 [==============================] - 6s 12ms/step - loss: 0.4771
Epoch 13/20
494/494 [===

In [8]:
test_df

,text,sense,honor,curse,despise,situation,antihuman,roughness,slaughter,strike_support,depression_rate
0,All the ghetto girls with hot cheeto bags,0.877843,0.997363,0.995188,0.851147,0.999204,0.504180,0.381056,0.213550,0.785664,2.791171e-01
1,Well we all know which people joins isis,0.467058,0.799236,0.525238,0.559148,0.528675,0.279252,0.082842,0.113763,0.492881,2.948185e-05
2,White Girl Wednesday 😍 URL,0.567875,0.650521,0.653007,0.399229,0.703661,0.653850,0.178289,0.000435,0.492881,2.828275e-08
3,"Same with the evil j€ws, they can go back to I...",0.787186,0.919010,0.960081,0.558150,0.690562,0.432708,0.265642,0.403168,0.720732,2.051049e-01
4,Why have all channels of communications in Kas...,0.765497,0.720054,0.767160,0.343499,0.554134,0.394411,0.284568,0.020697,0.270817,5.069130e-02
...,...,...,...,...,...,...,...,...,...,...,...
1749,"We had a ""best chick"". My husband's best frien...",0.072593,0.179267,0.242658,0.247456,0.302678,0.195999,0.033869,0.000971,0.492881,2.009373e-05
1750,White conservative males are statistically mor...,0.634883,0.342776,0.665943,0.454550,0.764795,0.125540,0.189315,0.054352,0.717645,6.614867e-03
1751,White liberals like to create white supremacis...,0.696091,0.800607,0.791712,0.886735,0.723604,0.084265,0.144857,0.021838,0.722567,3.272782e-01
1752,When a lady no longer gets frustrated or upset...,0.931834,0.491951,0.519997,0.205181,0.716309,0.620118,0.159258,0.000009,0.316153,4.265118e-08


In [9]:
test_df.drop(columns=['text'], inplace=True)
test_df

,sense,honor,curse,despise,situation,antihuman,roughness,slaughter,strike_support,depression_rate
0,0.877843,0.997363,0.995188,0.851147,0.999204,0.504180,0.381056,0.213550,0.785664,2.791171e-01
1,0.467058,0.799236,0.525238,0.559148,0.528675,0.279252,0.082842,0.113763,0.492881,2.948185e-05
2,0.567875,0.650521,0.653007,0.399229,0.703661,0.653850,0.178289,0.000435,0.492881,2.828275e-08
3,0.787186,0.919010,0.960081,0.558150,0.690562,0.432708,0.265642,0.403168,0.720732,2.051049e-01
4,0.765497,0.720054,0.767160,0.343499,0.554134,0.394411,0.284568,0.020697,0.270817,5.069130e-02
...,...,...,...,...,...,...,...,...,...,...
1749,0.072593,0.179267,0.242658,0.247456,0.302678,0.195999,0.033869,0.000971,0.492881,2.009373e-05
1750,0.634883,0.342776,0.665943,0.454550,0.764795,0.125540,0.189315,0.054352,0.717645,6.614867e-03
1751,0.696091,0.800607,0.791712,0.886735,0.723604,0.084265,0.144857,0.021838,0.722567,3.272782e-01
1752,0.931834,0.491951,0.519997,0.205181,0.716309,0.620118,0.159258,0.000009,0.316153,4.265118e-08


In [10]:
test_df = test_df * 4
test_df

,sense,honor,curse,despise,situation,antihuman,roughness,slaughter,strike_support,depression_rate
0,3.511371,3.989452,3.980751,3.404587,3.996817,2.016720,1.524225,0.854200,3.142657,1.116468e+00
1,1.868234,3.196944,2.100952,2.236594,2.114698,1.117007,0.331367,0.455051,1.971525,1.179274e-04
2,2.271501,2.602083,2.612027,1.596914,2.814644,2.615399,0.713156,0.001738,1.971525,1.131310e-07
3,3.148743,3.676038,3.840324,2.232601,2.762248,1.730831,1.062567,1.612674,2.882929,8.204197e-01
4,3.061987,2.880217,3.068638,1.373994,2.216536,1.577642,1.138273,0.082786,1.083269,2.027652e-01
...,...,...,...,...,...,...,...,...,...,...
1749,0.290373,0.717067,0.970630,0.989824,1.210711,0.783995,0.135476,0.003885,1.971525,8.037492e-05
1750,2.539531,1.371106,2.663773,1.818198,3.059181,0.502159,0.757259,0.217406,2.870578,2.645947e-02
1751,2.784366,3.202429,3.166848,3.546942,2.894415,0.337059,0.579430,0.087353,2.890266,1.309113e+00
1752,3.727338,1.967804,2.079989,0.820723,2.865234,2.480471,0.637032,0.000037,1.264612,1.706047e-07


In [11]:
test_df = test_df.round(3)
test_df

,sense,honor,curse,despise,situation,antihuman,roughness,slaughter,strike_support,depression_rate
0,3.511,3.989,3.981,3.405,3.997,2.017,1.524,0.854,3.143,1.116
1,1.868,3.197,2.101,2.237,2.115,1.117,0.331,0.455,1.972,0.000
2,2.272,2.602,2.612,1.597,2.815,2.615,0.713,0.002,1.972,0.000
3,3.149,3.676,3.840,2.233,2.762,1.731,1.063,1.613,2.883,0.820
4,3.062,2.880,3.069,1.374,2.217,1.578,1.138,0.083,1.083,0.203
...,...,...,...,...,...,...,...,...,...,...
1749,0.290,0.717,0.971,0.990,1.211,0.784,0.135,0.004,1.972,0.000
1750,2.540,1.371,2.664,1.818,3.059,0.502,0.757,0.217,2.871,0.026
1751,2.784,3.202,3.167,3.547,2.894,0.337,0.579,0.087,2.890,1.309
1752,3.727,1.968,2.080,0.821,2.865,2.480,0.637,0.000,1.265,0.000


In [12]:
test_df.to_csv('output.csv', index=False)